# Type structure and interface of lattice objects

The most defining feature of a **lattice** is the regular arrangement of **sites**, connected by **bonds** that are defined in **unitcells**. Lattices are considered objects with a finite extent in real space in the context of `LatticePhysics.jl`. They are represented by their own type, equipped with an abstract supertype as well as a shared interface.

The following tutorial aims on providing an overview over the type hierachy of lattices, introducing the abstract type `AbstractLattice`, as well as discussing the interface functions for this abstract type by showing examples for the concrete struct `Lattice`.

It is advised to be already familiar with the type structure of [unitcells](https://github.com/janattig/LatticePhysics_Tutorials/blob/master/basics/unitcells/unitcell_type_interface.ipynb), [sites](https://github.com/janattig/LatticePhysics_Tutorials/blob/master/basics/sites_bonds/site_type_interface.ipynb) and [bonds](https://github.com/janattig/LatticePhysics_Tutorials/blob/master/basics/sites_bonds/bond_type_interface.ipynb), as `AbstractLattice` will be parametric in these types.


In [ ]:
using LatPhysBase

### Abstract type `AbstractLattice{S,B,U}`

The abstract type that is supertype to all lattice implementations is `AbstractLattice`, parametric in three parameters `S`, `B` and `U`. These three parameters are the site type `S`, subtype of `AbstractSite` and the bond type `B`, subtype of `AbstractBond`, as well as the unitcell type `U` of the unitcell that was the building block of the lattice. Note that `U` is subtype of `AbstractUnitcell` and therefore is by itself parametric in two types `SU` and `BU` which are site and bond type of the unitcell respectively and can be different from `S` and `B` (of the lattice).

As the lattice type is parametric in these three types already, it is possible to dispatch lattice objects based on
- real space dimension
- Bravais lattice dimension
- label types of sites and bonds

This dispatch behavior allows for a compact but yet powerful solution in unifying syntax, e.g. `plotLattice` instead of two functions `plotLattice2D` and `plotLattice3D`.

In [ ]:
# this is a correct suptyping of lattice types
AbstractLattice{ Site{String,2}, Bond{String,2}, Unitcell{Site{Float64,2}, Bond{Float64,2}} } <: AbstractLattice

More on dispatch can be found in the dispatch [examples](https://github.com/janattig/LatticePhysics_Tutorials/blob/master/basics/lattices/lattice_dispatch.ipynb) for lattices. Note that in practice, the precise type structure of a lattice object is close to never of importance, it only plays a role in dispatch.

### Concrete type `Lattice{S,B} <: AbstractLattice{S,B}`

For all explicit lattice objects, a concrete lattice type is needed. Although every user could in principle implement its own concrete lattice type, a default implementation is provided within `LatticePhysics.jl`, the `mutable struct Lattice` implementation.

`Lattice` should be regarded as a subtype of `AbstractLattice` and can therefore be instatiated so it should be used whenever a concrete implementation is needed, i.e. whenever objects are created.

In [ ]:
Lattice{ Site{String,2}, Bond{String,2}, Unitcell{Site{Float64,2}, Bond{Float64,2}} } <: AbstractLattice{ Site{String,2}, Bond{String,2}, Unitcell{Site{Float64,2}, Bond{Float64,2}}}

Note: Despite seemingly being fixed to constructing objects of concrete type `Lattice{ Site{.}, Bond{.}, Unitcell{.} }`, the parametrization of `Lattice` allows for custom site and bond types to be inserted instead. For a user defined site type `MySite{L,D} <: AbstractSite{L,D}` (which might store more information than position and label), the type `Lattice{ MySite{.}, Bond{.}, Unitcell{.} }` can be constructed!




### Interface of `AbstractLattice` - how to access information correctly

In order to gain a benefit from having an abstract supertype `AbstractLattice`, one has to define a common interface that all concrete lattice types that are subtype of `AbstractLattice` have to implement. Then one can simply call this interface for a given (but unknown) lattice object that is subtype of `AbstractLattice` and rely on getting the correct results.

The interface of `AbstractLattice` contains various functions, which are explained in the following. Note, that nearly all functions are identical to the `AbstractUnitcell` interface, with a few small exceptions to take the differences of lattices and unitcells into account.


##### 1. The constructor

The constructor interface is used to create new objects of a specified lattice type. It is therefore demanding to pass the type explicitly, as well as further information regarding the lattice. 

The constructor syntax is
```
function newLattice(
            ::Type{L},
            lattice_vectors :: Vector{<:Vector{<:Real}},
            sites           :: Vector{S},
            bonds           :: Vector{B},
            unitcell        :: U
        ) :: L where{...}
```
This interface is overwritten by the concrete lattice type `Lattice` to
```
function newLattice(
            :: Type{Lattice{S,B,U}},
            lattice_vectors :: Vector{<:Vector{<:Real}},
            sites           :: Vector{S},
            bonds           :: Vector{B},
            unitcell        :: U
        ) :: Lattice{S,B,U} where {D,N,LS,LB,S<:AbstractSite{LS,D},B<:AbstractBond{LB,N}, SU,BU,U<:AbstractUnitcell{SU,BU}}
```


*Note: If you want a lattice for your calculations, the chances are high that you will construct a lattice from a unitcell that is already implemented with all site positions and bonds set correctly. For these pre-implemented lattices, check the pre-implemented lattices tutorial ([[notebook](https://github.com/janattig/LatticePhysics_Tutorials/blob/master/basics/lattices/pre-implemented_lattices.ipynb)]). Then, you can use a pre-implemented lattice building routine (like building a periodic LxL lattice) to generate a lattice out of it. More about this can be found in [this tutorial](https://github.com/janattig/LatticePhysics_Tutorials/blob/master/basics/lattices/lattice_building.ipynb)*

Using the constructor in constructing a lattice object could work as follows:

In [ ]:
# list of sites
site_list = Site{String,2}[
    newSite(Site{String,2}, [0,0], "mysite"),
    newSite(Site{String,2}, [0.5,0.5], "myothersite")
]

# list of bonds
bond_list = Bond{String,2}[
    newBond(Bond{String,2}, 1, 2, "mybond", (1,0)),
    newBond(Bond{String,2}, 2, 1, "mybond", (-1,0)),
    newBond(Bond{String,2}, 1, 2, "mybond", (0,1)),
    newBond(Bond{String,2}, 2, 1, "mybond", (0,-1)),
    newBond(Bond{String,2}, 1, 2, "mybond", (1,1)),
    newBond(Bond{String,2}, 2, 1, "mybond", (1,-1))
]

# list of lattice vectors
lattice_vectors = Vector{Float64}[
        Float64[1, 0],
        Float64[0, 1]
    ]

# construct the unitcell
uc = newUnitcell(
    Unitcell{Site{String,2},Bond{String,2}},
    lattice_vectors,
    site_list,
    bond_list
)




# construct the lattice (using the same sites and bonds, i.e. a 1x1 lattice)
lt = newLattice(
    Lattice{Site{String,2},Bond{String,2},Unitcell{Site{String,2},Bond{String,2}}},
    lattice_vectors,
    site_list,
    bond_list,
    uc
)

##### 2. Setter and getter of sites, bonds and lattice vectors

The basic type of information contained in lattices is
- list of sites
- list of bonds
- Bravais lattice vectors
- unitcell

There are interface functions for the lattice as setters and getters for these fields which differ in definition by a `!` to pronounce the similarity in language. Syntax is of the structure
```
julia> field = getter(lattice)

julia> setter!(lattice, field)
```
for examples, see below.

Note: As these functions are part of the lattice interface, they have to be implemented for each concrete type explicitly.

In [ ]:
# get list of all sites of the lattice
site_list = sites(lt)

# set the complete list of sites of the lattice
sites!(lt, site_list)

In [ ]:
# get list of all bonds of the lattice
bond_list = bonds(lt)

# set the complete list of bonds of the lattice
bonds!(lt, bond_list)

In [ ]:
# get the list of all lattice vectors of the Bravais lattice
lv = latticeVectors(lt)

# set the Bravais lattice vectors
latticeVectors!(lt, lv)

In [ ]:
# get the unitcell
uc = unitcell(lt)

# set the unitcell
unitcell!(lt, uc)

##### 3. Wrapper of sites and bonds for direct access

In many cases, one is interested in individual properties of the site and bond list respectively. For this purpose, short hand notation is defined on the level of `AbstractLattice`, s.t. it calls interface functions which had to be defined by the user. Therefore, this short hand notation does not have to be overwritten by each concrete type.

The short hand notation for sites and bonds contains
- functions to access the number of sites and bonds (e.g. for boundaries of for loops)
- getter for the i-th site or bond without obtaining the complete site or bond list

In [ ]:
# number of sites
println("lattice has ",  numSites(lt),  " site(s)")

# number of bonds
println("lattice has ",  numBonds(lt),  " bond(s)")

In [ ]:
# get site 1
println(  site(lt, 1)  )

# get bond 3
println(  bond(lt, 3)  )

##### 4. Bond list organized by origin / destination site

In some cases when an application needs bonds specifically connected to a certain site, having all bonds in one single list is not efficient. In these cases it might be easier to work with an ordered version of the bond list. This version is not a list of bonds, but rather a list of lists of bonds, where the `i`-th list either contains bonds terminating or originating from site `i`.

The functions to generate these lists are implemented on the `AbstractLattice` level again, i.e. only call interface functions and therefore are not needed to be implemented for every concrete type specifically.

For usage, see the examples below:

In [ ]:
# get a list of all bonds, sorted by origin site
organizedBondsFrom(lt)

In [ ]:
# get a list of all bonds, sorted by destination site
organizedBondsTo(lt)

##### 5. Direct access to certain Bravais lattice vectors

If one wants to directly access the Bravais lattice vectors individually, one can use the short hand notation getters `a1`, `a2`, `a3` to do so. These functions are simple wrappers of the form `ai(lt) = latticeVectors(lt)[i]` and work as demonstrated in the examples below.

These functions are again implemented using interface functions so that they dont have to be overwritten for every concrete site.

In [ ]:
# get the first Bravais lattice vector
a1(lt)